In [ ]:
%matplotlib inline
import os
import shutil
import pyemu
import matplotlib.pyplot as plt
import helpers

In [ ]:
working_d = "model_and_pest_files"
assert os.path.exists(working_d)

In [ ]:
pst = pyemu.Pst(os.path.join(working_d, "pest.pst"))
assert pst.nobs > pst.nnz_obs
assert pst.nnz_obs > 0

In [ ]:
pst.control_data.noptmax = -1
pst.pestpp_options["ies_num_reals"] = 30
pst.pestpp_options["ies_bad_phi_sigma"] = 1.75
# pst.pestpp_options["ies_n_iter_reinflate"] = [2, 999]
# pst.pestpp_options["ies_reinflate_num_reals"] = [50, -50]
# pst.pestpp_options["ies_multimodal_alpha"] = 0.99
# pst.pestpp_options.pop("ies_obs_en",None)
# pst.pestpp_options["ies_no_noise"] = True

pst.pestpp_options

In [ ]:
pst.write(os.path.join(working_d, "pest.pst"), version=2)

In [ ]:
master_dir = "master_ies_advanced"

In [ ]:
if os.path.exists(master_dir):
    shutil.rmtree(master_dir)
os.makedirs(master_dir)
pyemu.os_utils.start_workers(
    working_d,
    "pestpp-ies",
    "pest.pst",
    num_workers=10,
    worker_root=master_dir,
    master_dir=master_dir,
)

In [ ]:
noptmax = None

In [ ]:
helpers.plot_ies_timeseries(master_dir, noptmax=noptmax)

In [ ]:
tags = [
    "npf-k-layer1",
    "npf-k-layer5",
    "npf-k33-layer3",
    "sto-ss-layer1",
    "sto-sy-layer1",
]

In [ ]:
for tag in tags:
    helpers.plot_ies_properties(master_dir, tag, noptmax=noptmax)

In [ ]:
helpers.plot_ies_forecasts(master_dir, noptmax=noptmax)

In [ ]:
pst = pyemu.Pst(os.path.join(master_dir, "pest.pst"))

In [ ]:
for i in pst.ies.pcs.index.get_level_values(0).unique():
    print(i)
    lbnd = pst.ies.pcs.loc[i, "percent_at_near_lbound"]
    ubnd = pst.ies.pcs.loc[i, "percent_at_near_ubound"]
    fig, axes = plt.subplots(2, 1, figsize=(6, 6))
    lbnd.sort_values(inplace=False, ascending=False).iloc[:10].plot(
        kind="bar", ax=axes[0]
    )
    ubnd.sort_values(inplace=False, ascending=False).iloc[:10].plot(
        kind="bar", ax=axes[1]
    )
    axes[0].set_title("iteration {0}".format(i))
    plt.tight_layout()